## Tuning Model Parameters

In this exercise, you will optimise the parameters for a classification model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [1]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# Load the source data
csv = spark.read.csv('../../data/flights.csv', inferSchema=True, header=True)

# Select features and label
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 5).cast("Int").alias("label")))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a classification model

In [2]:
# Define the pipeline
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
lr = LogisticRegression(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[assembler, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. A simple way to do this is to use  **TrainValidationSplit** to evaluate each combination of parameters defined in a **ParameterGrid** against a subset of the training data in order to find the best performing parameters.

In [6]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 1, 10]).addGrid(lr.maxIter, [100,10,5]).build()

cv = CrossValidator(estimator=pipeline, evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid, numFolds=10)

import time
tic = time.time()
model = cv.fit(train)
toc = time.time()
print("Elapsed time ", toc-tic)

Elapsed time  4655.750661849976


### Test the Model
Now you're ready to apply the model to the test data.

In [4]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")

+--------------------+----------+--------------------+---------+
|            features|prediction|         probability|trueLabel|
+--------------------+----------+--------------------+---------+
|[1.0,1.0,10140.0,...|       0.0|[0.85464654181786...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.83316305615144...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.81736937036623...|        0|
|[1.0,1.0,10140.0,...|       1.0|[0.40329594997220...|        1|
|[1.0,1.0,10140.0,...|       0.0|[0.85491741697244...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.75833566921205...|        1|
|[1.0,1.0,10140.0,...|       0.0|[0.68135848216647...|        1|
|[1.0,1.0,10140.0,...|       0.0|[0.86159233512184...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.85492777856834...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.83709925109943...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.75329492258881...|        1|
|[1.0,1.0,10140.0,...|       0.0|[0.73770475962310...|        1|
|[1.0,1.0,10140.0,...|   

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [5]:
tp = float(predicted.filter("prediction == 1.0 AND trueLabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND trueLabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND trueLabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND trueLabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           45566.0|
|       FP|             133.0|
|       TN|          648211.0|
|       FN|          115835.0|
|Precision|0.9970896518523381|
|   Recall| 0.282315475120972|
+---------+------------------+

